### Implementing Sentiment Analysis for the Machine Hack Hackathon using the Tf-IDF model

#### Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
twitter_df = pd.read_csv("train.csv")

In [3]:
twitter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44100 entries, 0 to 44099
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         44100 non-null  int64 
 1   author     44100 non-null  object
 2   Review     44100 non-null  object
 3   Sentiment  44100 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.3+ MB


In [4]:
twitter_df.head(10)

,ID,author,Review,Sentiment
0,39467,rayinstirling,Today I'm working on my &quot;Quirky Q&quot; c...,2
1,30154,DirtyRose17,@ShannonElizab dont ya know? people love the h...,1
2,16767,yoliemichelle,ughhh rejected from the 09 mediation program. ...,0
3,9334,jayamelwani,@petewentz im so jealous. i want an octo drive,0
4,61178,aliisanoun,I remember all the hype around this movie when...,0
5,54688,empressjazzy1,I liked this quite a bit but I have friends th...,2
6,34838,lorrief,loving that spring definitely seems to be here...,2
7,28520,GE0RGIE,@jeg007jeg yay coutch:couch,2
8,31974,BrandonCarlson,"Working on the store's Facebook group, getting...",2
9,14323,allshookup,@falselove OH THAT'S GOOD! My top 4 are: The H...,0


It can be seen that data cleaning is required. This is because:
- Some reviews have the user's info seen as: @username
- There are words which do not necessarily contribute to the sentiment expressed.

In [9]:
import re

In [5]:
import nltk

In [6]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [7]:
corpus = []
snow_stemmer = SnowballStemmer(language='english')

In [8]:
review_series = twitter_df["Review"]

#### Starting with the text cleaning and following these steps:
1) Converting all data to lower case<br>
2) Removing punctuations<br>
3) Removing HTML tags<br>
4) Removing stopwords<br>
5) Performing stemming<br>
6) Performing lemmatization